In [1]:
import pandas
from os import listdir

def getFiles(s):
    return map(lambda x: f"{s}/{x}", listdir(s))
# filenames = [*getFiles("out-results"), *getFiles("out-shuffle"), *getFiles("out-structure-10-10")]
filenames = getFiles("out")
files = [pandas.read_csv(file, sep=';')
         for file in filenames if 'scores-' not in file]

df = pandas.concat(files)

# rest_filenames = getFiles("out-rerun")
# rest_files = [pandas.read_csv(file, sep=';')
#          for file in rest_filenames if 'scores-' not in file]
# df2 = pandas.concat(rest_files)
# df2 = df2.loc[~df2['Algorithm'].isin(['StructuredUnification', 'StructuredSplitUnification', 'SplitTypeUnification', 'SplitTypeSplitUnification', 'SimpleUnification', 'SimpleSplitUnification'])]

In [2]:
combined = df
trimmed = combined[['Algorithm', 'Ranking Factor', 'Size Factor', 'Index', 'Line', 'Character', 'Keywords Before', 'File Name']]
trimmed.loc[trimmed['Ranking Factor'].isnull(), 'Ranking Factor'] = 0
trimmed.loc[trimmed['Size Factor'].isnull(), 'Size Factor'] = 0
groups = trimmed.groupby(['Algorithm', 'Ranking Factor', 'Size Factor'])
# groups = trimmed.groupby(['File Name', 'Algorithm', 'Ranking Factor', 'Size Factor'])

In [7]:
fullDump = True  # Do we take all the results for Type Structure, or just the best?
prefix0 = False  # Are scores calculated only for a short prefix?
applyrewrite = False  # Do we split on appl and rewrite?
allErrors = False  # Do we include errors in scoring? These are usually caused by every algorithm, so it's not very interesting

In [8]:
from collections import defaultdict as dd
from itertools import product
allresults: dict[tuple, dict[int, tuple[int, dict[int, int]]]] = {}  # groupname -> lemmaPosition -> (count, (resultIndex -> count))
data = []
allWords = set()
allCounts = {}
for (groupName, frame), keyword in product(groups, ['apply', 'rewrite'] if applyrewrite else [None]):
    # if 'Simple' not in groupName[0]: continue
    if not fullDump and groupName[1] not in [0,5]: continue
    if not fullDump and groupName[2] not in [0,5]: continue
    # if (not (
    #     "Structure" in groupName[0]
    #     and groupName[1] == 2
    #     and groupName[2] == 5)): continue
    line = -1
    ch = -1
    lemmaPosition = -1
    thisdata = dd(lambda: dd(int))
    positionCounts = dd(int)
    allresults[groupName] = {}
    for resultIndex, l, c, kw in frame[['Index', 'Line', 'Character', 'Keywords Before']].values:
        allWords.add(kw)
        if not allErrors and resultIndex == -2: continue
        # if (type(kw) != str): continue
        if line != l:
            line = l
            ch = c
            lemmaPosition = 0
        if ch != c:
            if ch + 1 == c:
                lemmaPosition += 1
            else:
                lemmaPosition = 0
            ch = c
        if not keyword or (type(kw) == str and keyword in kw):
            thisdata[lemmaPosition][resultIndex] += 1
            positionCounts[lemmaPosition] += 1
    
    grades = dd(int)
    for lemmaPosition, results in thisdata.items():
        attempts = positionCounts[lemmaPosition]
        allCounts.update({lemmaPosition: attempts})
        allresults[groupName].update({lemmaPosition: (attempts, results)})
        grades[lemmaPosition] = sum(map(lambda j: results[j] / (j + 1), range(10))) / attempts
    a = sum(map(lambda x: grades[x] / (x + 1), range(1 if prefix0 else 9)))
    b = sum(map(lambda x: 1 / (x+1), range(1 if prefix0 else 9)))
    if b == 0:
        print(groupName, a, "failed")
        continue
    score = 100 * ( a / b )
    data.append((*groupName, *([keyword] if applyrewrite else []), round(score, 2)))
result_df = pandas.DataFrame(data, columns=['Algorithm', 'Atomic Bias', 'Size Bias', *(['Keyword'] if applyrewrite else []), 'Score'])
result_df.sort_values(by=['Score'], inplace=True, ascending=False)
result_df

,Algorithm,Atomic Bias,Size Bias,Score
18,StructuredSplitUnification,5.0,5.0,51.13
15,StructuredSplitUnification,2.0,5.0,51.06
37,StructuredUnification,5.0,5.0,51.05
34,StructuredUnification,2.0,5.0,50.98
27,StructuredTypeEvaluation,5.0,5.0,50.87
28,StructuredTypeEvaluation,10.0,10.0,50.84
24,StructuredTypeEvaluation,2.0,5.0,50.83
17,StructuredSplitUnification,5.0,2.0,50.76
36,StructuredUnification,5.0,2.0,50.68
12,StructuredSplitUnification,1.0,5.0,50.66


In [5]:
good_names = {
'Basic': 'Baseline',
'Shuffle': 'Shuffle',
'SimpleTypeIntersection': 'Type Intersection',
'SimpleUnification': 'Type Intersection with Unification',
'SimpleSplitUnification': 'Type Intersection with Split Unification',
'SplitTypeIntersection': 'Split Type Intersection',
'SplitTypeUnification': 'Split Type Intersection with Unification',
'SplitTypeSplitUnification': 'Split Type Intersection with Split Unification',
'StructuredTypeEvaluation': 'Type Structure',
'StructuredUnification': 'Type Structure with Unification',
'StructuredSplitUnification': 'Type Structure with Split Unification',
'ShuffleSplitUnification': 'Shuffle with Split Unification',
'ShuffleUnification': 'Shuffle with Unification',
}

In [ ]:
tableLength = 8
foreach = {}
for (algo, rank, size), v in allresults.items():
    lmao = []
    foreach[(algo, rank, size)] = lmao
    for lemmaPosition, (amount, results) in v.items():
        if lemmaPosition > tableLength:
            break
        first = results[0]
        second = results[1]
        third = results[2]
        rest = sum(map(lambda x: results[x], range(3, 10)))
        fail = amount - sum([first, second, third, rest])
        passed = amount - fail
        lmao.append([*map(lambda x: round(x * 100 / amount, 2), [first, second, third, rest, passed, fail])])
    print(r'\begin{table}[h]')
    print(r'\footnotesize')
    print('\\begin{tabular}{|r|', *map(lambda x: "r", range(tableLength + 1)), r'|}\hline', sep="")
    # print(f'\multicolumn{{{tableLength + 2}}}{{l}}{{\\normalsize', k, '}', '\\\\', '\\hline')
    print("Prefix", *map(lambda x: rf'\ctd{{{x}}}', range(tableLength)), rf'\ctdl{{{tableLength}}}', sep=' & ', end='\\\\\\hline\n')
    for x, n in enumerate(["1st","2nd","3rd","4-10th","1-10th","Fail"]):
        print(n, *map(lambda l: f'{l[x]:.1f}\\%', lmao), sep=' & ', end=r"\\")
        if x == 3:
            print(r"\hline", end="")
        print()
    print('\\hline\\end{tabular}')
    print(r'\vspace{0.4em}')
    params = '' if rank == 0 else f', with Atomic Bias {int(rank)} and Size Bias {int(size)}'
    print(rf'\caption{{Test results from {good_names[algo]}{params}, showing for each prefix length how large a proportion of results appeared at each position in the completions.}}')
    print(r'\label{tab:appendix-results-', algo.lower(), ('-' + str(int(rank)) + '-' + str(int(size)) if rank else ''), '}', sep='')
    print(r'\end{table}')
    print()

In [ ]:
d = []
for r55, r25 in zip(foreach[('StructuredSplitUnification', 5.0, 5.0)], foreach[('StructuredUnification', 5.0, 5.0)]):
    d.append([*map(lambda x: x[0] - x[1], zip(r55, r25))])
for x, n in enumerate(["1st","2nd","3rd","4-10th","1-10th","Fail"]):
    print(n, *map(lambda l: f'{l[x]:+.1f}', d), sep=' & ', end=r"\\")
    if x == 3:
        print(r"\hline", end="")
    print()

In [ ]:
matrix = [
    ['Basic', None],
    ['Shuffle', 'ShuffleSplitUnification'],
    ['SimpleTypeIntersection', 'SimpleSplitUnification'],
    ['SplitTypeIntersection', 'SplitTypeSplitUnification'],
    ['StructuredTypeEvaluation', 'StructuredSplitUnification'],
] if applyrewrite else [
    ['Basic', None, None],
    ['Shuffle', 'ShuffleUnification', 'ShuffleSplitUnification'],
    ['SimpleTypeIntersection', 'SimpleUnification', 'SimpleSplitUnification'],
    ['SplitTypeIntersection', 'SplitTypeUnification', 'SplitTypeSplitUnification'],
    ['StructuredTypeEvaluation', 'StructuredUnification', 'StructuredSplitUnification'],
]

print(r'\begin{table}[h]')
print(r"\begin{tabular}{|r|l|p{2cm}|p{2cm}|}\hline")
if applyrewrite:
    print(r"Algorithm & apply & rewrite & apply, Split Unification & rewrite, Split Unification \\\hline")
else:
    print(r"Algorithm & Score & Score, Unification & Score, Split Unification \\\hline")

for row in matrix:
    print(good_names[row[0]], end=' ')
    for algo in row:
        if algo is None:
            print('&   -  ', end=' ')
            continue
        if applyrewrite:
            filt = result_df.loc[result_df['Algorithm'] == algo, ['Algorithm','Keyword', 'Score']]
            print('&', filt.loc[result_df['Keyword'] == 'apply', 'Score'].values[0], end=' ')
            print('&', filt.loc[result_df['Keyword'] == 'rewrite', 'Score'].values[0], end=' ')
        else:
            print('&', result_df.loc[result_df['Algorithm'] == algo, 'Score'].values[0], end=' ')
    print(r'\\')
print(r'\hline\end{tabular}')
print(r'\vspace{0.4em}')
print(r'\caption{Summary of the experiment results. Parameters for Type Structure are a ranking factor of 5 and size factor of 5 in all three cases, as it performed the best.}')
print(r'\label{tab:score-summary}')
print(r'\end{table}')

In [6]:
def getFiles(s):
    return map(lambda x: f"{s}/{x}", listdir(s))
filenames = getFiles("out")
time_files = [pandas.read_csv(file, sep=';')
         for file in filenames if 'scores' in file]
time_df = pandas.concat(time_files)
time_df = time_df[['Algorithm', 'Ranking Factor', 'Size Factor', 'Time']].groupby(['Algorithm', 'Ranking Factor', 'Size Factor']).sum().apply(lambda x: round(x / allCounts[0]))
time_df

,,,Time
Algorithm,Ranking Factor,Size Factor,
Basic,0,0,675.0
Shuffle,0,0,686.0
ShuffleSplitUnification,0,0,688.0
ShuffleUnification,0,0,687.0
SimpleSplitUnification,0,0,700.0
SimpleTypeIntersection,0,0,709.0
SimpleUnification,0,0,698.0
SplitTypeIntersection,0,0,716.0
SplitTypeSplitUnification,0,0,709.0


In [ ]:
matrix = [
    [('Basic', 0, 0), None, None],
    [('Shuffle', 0, 0), ('ShuffleUnification', 0, 0), ('ShuffleSplitUnification', 0, 0)],
    [('SimpleTypeIntersection', 0, 0), ('SimpleUnification', 0, 0), ('SimpleSplitUnification', 0, 0)],
    [('SplitTypeIntersection', 0, 0), ('SplitTypeUnification', 0, 0), ('SplitTypeSplitUnification', 0, 0)],
    [('StructuredTypeEvaluation', 5, 5), ('StructuredUnification', 5, 5), ('StructuredSplitUnification', 5, 5)],
]

print(r'\begin{table}[h]')
print(r"\begin{tabular}{|r|l|p{2cm}|p{2cm}|}\hline")

print(r"Algorithm & Time (ms) & Time, Unification & Time, Split Unification \\\hline")

for row in matrix:
    print(good_names[row[0][0]], end=' ')
    for algo in row:
        if algo is None:
            print('&   -  ', end=' ')
            continue
        print('&', int(time_df.loc[algo].values[0]), end=' ')
    print(r'\\')
print(r'\hline\end{tabular}')
print(r'\vspace{0.4em}')
print(r'\caption{Summary of the experiment results. Parameters for Type Structure are a ranking factor of 5 and size factor of 5 in all three cases, as it performed the best.}')
print(r'\label{tab:score-summary}')
print(r'\end{table}')

In [ ]:
x = result_df[result_df['Algorithm'] == 'StructuredSplitUnification']
ressss = [
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
]
inds = [1,2,5,10]
for algo, rf, sf, s in x.values:
    ressss[inds.index(sf)][inds.index(rf)] = s
print('     ', *inds[:3], sep='   &   ', end=r'   \\\hline')
print()
for i in range(3):
    print('~~', inds[i], '~~', end='')
    for j in range(3):
        print(' &', f'{ressss[i][j]:.2f}', end='')
    print(r' \\')